In [20]:
// Setup: Imports und Konfiguration
import { load } from "https://deno.land/std@0.224.0/dotenv/mod.ts";
import { privateKeyToAccount } from "npm:viem@2/accounts";
import { createPublicClient, http, formatUnits, parseUnits } from "npm:viem@2";
import { optimism, optimismSepolia } from "npm:viem@2/chains";

// Load environment variables
const env = await load({ export: true });

const PRIVATE_KEY = env.TEST_WALLET_PRIVATE_KEY;
const PAY_TO_ADDRESS = env.NFT_WALLET_PUBLIC_KEY;

// Create account from private key
const account = privateKeyToAccount(`0x${PRIVATE_KEY}`);

console.log("🚀 x402 Facilitator Demo (TypeScript/Deno)");
console.log(`   Payer Address: ${account.address}`);
console.log(`   Recipient Address: ${PAY_TO_ADDRESS}`);

🚀 x402 Facilitator Demo (TypeScript/Deno)
   Payer Address: 0x553179556FC2A39e535D65b921e01fA995E79101
   Recipient Address: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C


## Network-Konfiguration

Wähle zwischen **Testnet** (Optimism Sepolia) und **Mainnet** (Optimism mit echtem Geld).

⚠️ **WARNUNG**: Bei `USE_MAINNET = true` wird echtes USDC verwendet!

In [ ]:
// ⚠️ NETWORK SELECTION - Change this to switch between testnet and mainnet
const USE_MAINNET = false;  // Set to true for Optimism Mainnet with REAL MONEY

// Network configuration
const config = USE_MAINNET ? {
    chain: optimism,
    chainId: 10,
    caip2Network: "eip155:10" as const,
    networkName: "Optimism Mainnet",
    usdcAddress: "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85" as `0x${string}`,
    usdcName: "USD Coin",
    rpcUrl: "https://mainnet.optimism.io"
} : {
    chain: optimismSepolia,
    chainId: 11155420,
    caip2Network: "eip155:11155420" as const,
    networkName: "Optimism Sepolia (Testnet)",
    usdcAddress: "0x5fd84259d66Cd46123540766Be93DFE6D43130D7" as `0x${string}`,
    usdcName: "USDC",
    rpcUrl: "https://sepolia.optimism.io"
};

// Payment configuration
const PAYMENT_AMOUNT = "20000";  // $0.02 USDC (6 decimals)

if (USE_MAINNET) {
    console.log(`\n🚨 WARNING: Using REAL MONEY on ${config.networkName}!`);
} else {
    console.log(`\n🧪 Using testnet: ${config.networkName}`);
}

console.log(`   Chain ID: ${config.chainId}`);
console.log(`   CAIP-2 Network: ${config.caip2Network}`);
console.log(`   USDC Address: ${config.usdcAddress}`);
console.log(`   USDC Name: ${config.usdcName}`);
console.log(`\n💰 Payment Amount: ${PAYMENT_AMOUNT} (${Number(PAYMENT_AMOUNT) / 1e6} USDC)`);


🚨 WARNING: Using REAL MONEY on Optimism Mainnet!
   Chain ID: 10
   CAIP-2 Network: eip155:10
   USDC Address: 0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85
   USDC Name: USD Coin

💰 Payment Amount: 20000 (0.02 USDC)


## Facilitator Endpoints

Konfiguriere die Facilitator-URL (lokal oder deployed):

In [22]:
// Facilitator endpoint configuration
// const FACILITATOR_URL = "https://facilitator.fretchen.eu";
const FACILITATOR_URL = "http://localhost:8080";

const VERIFY_URL = `${FACILITATOR_URL}/verify`;
const SETTLE_URL = `${FACILITATOR_URL}/settle`;
const SUPPORTED_URL = `${FACILITATOR_URL}/supported`;

console.log("🚀 x402 Facilitator Endpoints:");
console.log(`   Base: ${FACILITATOR_URL}`);
console.log(`   Verify: ${VERIFY_URL}`);
console.log(`   Settle: ${SETTLE_URL}`);
console.log(`   Supported: ${SUPPORTED_URL}`);

🚀 x402 Facilitator Endpoints:
   Base: http://localhost:8080
   Verify: http://localhost:8080/verify
   Settle: http://localhost:8080/settle
   Supported: http://localhost:8080/supported


## Test /supported Endpoint

Prüfe welche Networks und Schemes der Facilitator unterstützt:

In [23]:
// Test the /supported endpoint
const supportedResponse = await fetch(SUPPORTED_URL);
const supported = await supportedResponse.json();

console.log(`Status Code: ${supportedResponse.status}`);
console.log(JSON.stringify(supported, null, 2));

console.log(`\n✅ Supported Networks:`);
for (const kind of supported.kinds) {
    console.log(`   - ${kind.network} (${kind.scheme} scheme, x402 v${kind.x402Version})`);
}

// Show signers (facilitator addresses per network)
if (supported.signers && Object.keys(supported.signers).length > 0) {
    console.log(`\n🔐 Facilitator Signers:`);
    for (const [pattern, addresses] of Object.entries(supported.signers)) {
        console.log(`   ${pattern}: ${(addresses as string[]).join(", ")}`);
    }
}

Status Code: 200
{
  "kinds": [
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:10"
    },
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:11155420"
    }
  ],
  "extensions": [
    {
      "name": "recipient_whitelist",
      "description": "Payment recipients must be authorized through smart contract whitelist. Clients can verify authorization by calling isAuthorizedAgent(address) on the contracts below.",
      "contracts": {
        "eip155:10": [
          {
            "name": "GenImNFTv4",
            "address": "0x80f95d330417a4acEfEA415FE9eE28db7A0A1Cdb",
            "method": "isAuthorizedAgent(address)"
          },
          {
            "name": "LLMv1",
            "address": "0x833F39D6e67390324796f861990ce9B7cf9F5dE1",
            "method": "isAuthorizedAgent(address)"
          }
        ]
      }
    }
  ],
  "signers": {
    "eip155:*": [
      "0x0000000000000000000000000000000000000000"
    ]
  }
}

✅ 

## x402 Client Setup mit offiziellen Packages

Hier nutzen wir die offiziellen `@x402/core` und `@x402/evm` Packages:

In [24]:
// Import x402 packages (same as genimg_x402_buyer.ipynb)
import { x402Client } from "npm:@x402/fetch@^2.0.0";
import { registerExactEvmScheme } from "npm:@x402/evm@^2.0.0/exact/client";

// Create x402 client
const client = new x402Client();

// Register EVM scheme - this is the simplified API!
// Just pass the viem account directly, no wrapper needed
registerExactEvmScheme(client, { signer: account });

console.log("✅ x402 Client configured");
console.log(`   Signer address: ${account.address}`);
console.log("   Registered scheme: exact (eip155:*)");

✅ x402 Client configured
   Signer address: 0x553179556FC2A39e535D65b921e01fA995E79101
   Registered scheme: exact (eip155:*)


## Create Payment Payload

Erstelle einen Payment Payload mit der x402 Client API:

In [25]:
// Build PaymentRequirements (what the server would send in a 402 response)
const paymentRequirements: PaymentRequirements = {
    scheme: "exact",
    network: config.caip2Network,
    amount: PAYMENT_AMOUNT,
    asset: config.usdcAddress,
    payTo: PAY_TO_ADDRESS,
    maxTimeoutSeconds: 3600,
    extra: {
        name: config.usdcName,
        version: "2"
    }
};

// Build a mock PaymentRequired response (normally from server's 402 response)
const paymentRequired = {
    x402Version: 2,
    accepts: [paymentRequirements],
    resource: {
        url: "https://example.com/resource",
        description: "Test payment",
        mimeType: "application/json"
    },
    extensions: {}
};

console.log("📝 Payment Requirements:");
console.log(JSON.stringify(paymentRequirements, null, 2));

📝 Payment Requirements:
{
  "scheme": "exact",
  "network": "eip155:10",
  "amount": "20000",
  "asset": "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85",
  "payTo": "0xAAEBC1441323B8ad6Bdf6793A8428166b510239C",
  "maxTimeoutSeconds": 3600,
  "extra": {
    "name": "USD Coin",
    "version": "2"
  }
}


In [26]:
// Create payment payload using x402 client
// This handles all the EIP-712 signing internally!
const paymentPayload: PaymentPayload = await client.createPaymentPayload(paymentRequired as any);

console.log("✅ Payment Payload created:");
console.log(JSON.stringify(paymentPayload, null, 2));

console.log(`\n🔐 Signature Details:`);
console.log(`   Network: ${config.networkName}`);
console.log(`   From: ${paymentPayload.payload?.authorization?.from}`);
console.log(`   To: ${paymentPayload.payload?.authorization?.to}`);
console.log(`   Value: ${paymentPayload.payload?.authorization?.value}`);
console.log(`   Signature: ${paymentPayload.payload?.signature?.slice(0, 42)}...`);

✅ Payment Payload created:
{
  "x402Version": 2,
  "payload": {
    "authorization": {
      "from": "0x553179556FC2A39e535D65b921e01fA995E79101",
      "to": "0xAAEBC1441323B8ad6Bdf6793A8428166b510239C",
      "value": "20000",
      "validAfter": "1766742823",
      "validBefore": "1766747023",
      "nonce": "0x3c1b0149923b057868b8255cfa69ba971226ae733d4f2ff220d01e5c4a0fe85d"
    },
    "signature": "0x8eef9c4ecdbed32cabce9477b1873eb4addc5da54f67678d27833d74a9bae98a142f4dcf13e50f7d90b40485971be18ad490a2ae087ce27dbb2294f57cd6f2f31b"
  },
  "extensions": {},
  "resource": {
    "url": "https://example.com/resource",
    "description": "Test payment",
    "mimeType": "application/json"
  },
  "accepted": {
    "scheme": "exact",
    "network": "eip155:10",
    "amount": "20000",
    "asset": "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85",
    "payTo": "0xAAEBC1441323B8ad6Bdf6793A8428166b510239C",
    "maxTimeoutSeconds": 3600,
    "extra": {
      "name": "USD Coin",
      "version": "2"

## Check USDC Balances

Vor dem Settlement prüfen wir die USDC Balances:

In [27]:
// Create public client for balance checks
const publicClient = createPublicClient({
    chain: config.chain,
    transport: http(config.rpcUrl)
});

// USDC ERC-20 balanceOf ABI
const erc20Abi = [
    {
        inputs: [{ name: "account", type: "address" }],
        name: "balanceOf",
        outputs: [{ name: "", type: "uint256" }],
        stateMutability: "view",
        type: "function"
    }
] as const;

// Check balances
const payerBalance = await publicClient.readContract({
    address: config.usdcAddress,
    abi: erc20Abi,
    functionName: "balanceOf",
    args: [account.address]
});

const recipientBalance = await publicClient.readContract({
    address: config.usdcAddress,
    abi: erc20Abi,
    functionName: "balanceOf",
    args: [PAY_TO_ADDRESS]
});

console.log(`💰 USDC Balances on ${config.networkName}:`);
console.log(`   Payer (${account.address.slice(0, 10)}...): ${formatUnits(payerBalance, 6)} USDC`);
console.log(`   Recipient (${PAY_TO_ADDRESS.slice(0, 10)}...): ${formatUnits(recipientBalance, 6)} USDC`);

if (payerBalance < BigInt(PAYMENT_AMOUNT)) {
    console.log(`\n⚠️ WARNING: Payer has insufficient USDC balance!`);
    console.log(`   Required: ${formatUnits(BigInt(PAYMENT_AMOUNT), 6)} USDC`);
}

💰 USDC Balances on Optimism Mainnet:
   Payer (0x55317955...): 0 USDC
   Recipient (0xAAEBC144...): 0.95 USDC

⚠️ WARNING: Payer has insufficient USDC balance!
   Required: 0.02 USDC


## Verify Payment

Sende den Payment Payload zur Verifizierung an den Facilitator:

In [28]:
// Build verify request
const verifyRequest = {
    paymentPayload: paymentPayload,
    paymentRequirements: paymentRequirements
};

console.log("🔍 Sending verification request...");

const verifyResponse = await fetch(VERIFY_URL, {
    method: "POST",
    headers: { "Content-Type": "application/json" },
    body: JSON.stringify(verifyRequest)
});

const verifyResult = await verifyResponse.json();

console.log(`\n📦 Verify Response (Status ${verifyResponse.status}):`);
console.log(JSON.stringify(verifyResult, null, 2));

if (verifyResult.isValid) {
    console.log(`\n✅ Payment signature is VALID!`);
    console.log(`   Payer: ${verifyResult.payer}`);
} else {
    console.log(`\n❌ Payment signature is INVALID!`);
    console.log(`   Reason: ${verifyResult.invalidReason}`);
}

🔍 Sending verification request...

📦 Verify Response (Status 200):
{
  "isValid": false,
  "invalidReason": "insufficient_funds",
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101"
}

❌ Payment signature is INVALID!
   Reason: insufficient_funds


## Settle Payment

Führe das Settlement on-chain aus:

In [29]:
// Attempt settlement
console.log(`\n💸 Attempting Settlement...`);
console.log(`   Network: ${config.networkName}`);
console.log(`   Payment: $${Number(PAYMENT_AMOUNT) / 1e6} USDC → ${PAY_TO_ADDRESS}`);

if (USE_MAINNET) {
    console.log(`\n🚨 WARNING: This will execute a REAL transaction with REAL MONEY!`);
    // In a real notebook, you'd have user confirmation here
}

const settleResponse = await fetch(SETTLE_URL, {
    method: "POST",
    headers: { "Content-Type": "application/json" },
    body: JSON.stringify(verifyRequest)
});

const settleResult = await settleResponse.json();

console.log(`\n📦 Settle Response (Status ${settleResponse.status}):`);
console.log(JSON.stringify(settleResult, null, 2));

let txHash: string | null = null;

if (settleResult.success) {
    txHash = settleResult.transaction;
    console.log(`\n🎉 Settlement successful!`);
    console.log(`   Transaction: ${txHash}`);
    console.log(`   Network (Response): ${settleResult.network}`);
} else {
    console.log(`\n❌ Settlement failed`);
    console.log(`   Reason: ${settleResult.errorReason || "unknown"}`);
}


💸 Attempting Settlement...
   Network: Optimism Mainnet
   Payment: $0.02 USDC → 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

🚨 WARNING: This will execute a REAL transaction with REAL MONEY!

📦 Settle Response (Status 200):
{
  "success": false,
  "errorReason": "insufficient_funds",
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101",
  "transaction": "",
  "network": "eip155:10"
}

❌ Settlement failed
   Reason: insufficient_funds


## Block Explorer & Cross-Chain Validation

Wichtig: Wir nutzen das `network` Feld aus der Server-Response, nicht lokale Variablen!

In [30]:
// Display transaction on block explorer
// ✅ WICHTIG: Wir nutzen das 'network' Feld aus der Server-Response!

if (txHash) {
    const responseNetwork = settleResult.network || "";
    
    // Cross-Chain Validation: Does response match expected network?
    if (responseNetwork && responseNetwork !== config.caip2Network) {
        console.log(`⚠️ WARNING: Network mismatch!`);
        console.log(`   Expected: ${config.caip2Network} (${config.networkName})`);
        console.log(`   Response: ${responseNetwork}`);
        console.log(`   Transaction may have been executed on a different network!`);
    }
    
    // Block Explorer URL mapping from CAIP-2 network
    const networkToExplorer: Record<string, string> = {
        "eip155:10": "https://optimistic.etherscan.io/tx/",
        "eip155:11155420": "https://sepolia-optimism.etherscan.io/tx/",
        "eip155:8453": "https://basescan.org/tx/",
        "eip155:84532": "https://sepolia.basescan.org/tx/",
    };
    
    const explorerBase = networkToExplorer[responseNetwork] || networkToExplorer[config.caip2Network];
    const explorerUrl = explorerBase ? `${explorerBase}${txHash}` : null;
    
    console.log(`\n🔍 Block Explorer:`);
    if (explorerUrl) {
        console.log(`   ${explorerUrl}`);
    }
    console.log(`\n📊 Transaction Details:`);
    console.log(`   • Hash: ${txHash}`);
    console.log(`   • Network (Response): ${responseNetwork}`);
    console.log(`   • Network (Expected): ${config.caip2Network}`);
    console.log(`   • Token: USDC at ${config.usdcAddress}`);
} else {
    console.log(`\n⚠️ No transaction hash available`);
}


⚠️ No transaction hash available


## Cross-Chain Balance Verification

Nach dem Settlement prüfen wir die Balances auf **beiden** Networks:

In [31]:
// Cross-Chain Balance Verification
console.log(`🔍 Cross-Chain Balance Check for Recipient: ${PAY_TO_ADDRESS}`);
console.log(`   Expected Settlement Network: ${config.caip2Network}\n`);

const networks = [
    {
        id: "eip155:10",
        name: "Optimism Mainnet",
        chain: optimism,
        rpc: "https://mainnet.optimism.io",
        usdc: "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85" as `0x${string}`
    },
    {
        id: "eip155:11155420", 
        name: "Optimism Sepolia",
        chain: optimismSepolia,
        rpc: "https://sepolia.optimism.io",
        usdc: "0x5fd84259d66Cd46123540766Be93DFE6D43130D7" as `0x${string}`
    }
];

for (const network of networks) {
    try {
        const client = createPublicClient({
            chain: network.chain,
            transport: http(network.rpc)
        });
        
        const balance = await client.readContract({
            address: network.usdc,
            abi: erc20Abi,
            functionName: "balanceOf",
            args: [PAY_TO_ADDRESS]
        });
        
        const balanceUsdc = formatUnits(balance, 6);
        
        // Mark expected network
        let marker = network.id === config.caip2Network ? "← Expected" : "";
        // Mark if response showed different network
        if (settleResult?.network === network.id && network.id !== config.caip2Network) {
            marker = "← ⚠️ SETTLEMENT HERE!";
        }
        
        console.log(`   ${network.name} (${network.id}): $${balanceUsdc} USDC ${marker}`);
    } catch (e) {
        console.log(`   ${network.name}: Error - ${e}`);
    }
}

console.log(`\n💡 Compare balances before and after settlement`);
console.log(`   to verify funds arrived on the correct network.`);

🔍 Cross-Chain Balance Check for Recipient: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
   Expected Settlement Network: eip155:10

   Optimism Mainnet (eip155:10): $0.95 USDC ← Expected
   Optimism Sepolia (eip155:11155420): $1.271 USDC 

💡 Compare balances before and after settlement
   to verify funds arrived on the correct network.


## Summary

Dieses Notebook demonstriert den kompletten x402 v2 Payment Flow:

1. ✅ **Setup**: x402Client mit ExactEvmScheme registriert
2. ✅ **Payment Creation**: Automatische EIP-712 Signatur via `createPaymentPayload()`
3. ✅ **Verification**: Signatur beim Facilitator verifiziert
4. ✅ **Settlement**: On-chain Transaktion ausgeführt
5. ✅ **Cross-Chain Validation**: Network aus Response validiert

**Vorteile der TypeScript/x402 Version:**
- Typsicherheit durch TypeScript
- Offizielle x402 Packages von Coinbase
- Automatisches Payment-Handling
- Korrekte EIP-712 Domain-Konfiguration durch die Library